In [14]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import numpy as np
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import pickle
import mdtraj as md
import itertools
from perses.tests.utils import compute_potential_components

conducting subsequent work with the following platform: CUDA


## test the energy discrepancy of the alanine dipeptide in solvent with CutoffPeriodic

In [2]:
from openmmtools.testsystems import AlanineDipeptideExplicit
ala = AlanineDipeptideExplicit(useDispersionCorrection=False, nonbondedMethod=app.CutoffPeriodic)

In [4]:
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]

In [5]:
ala.system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b2e797f7a80> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b2e43a236c0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b2e43a23060> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b2e43a23db0> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x2b2e43a23930> >]

In [6]:
ala.system.removeForce(4) #remove CMM

In [7]:
rtf = RESTTopologyFactory(system = ala.system, solute_region=rest_atoms, use_dispersion_correction=False)

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.


In [8]:
ss = SamplerState(positions=ala.positions)

In [9]:
og_thermostate = ThermodynamicState(ala.system, temperature=300*unit.kelvin)
og_integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
og_context = og_thermostate.create_context(og_integrator)
ss.apply_to_context(og_context)
ss.update_from_context(og_context)
print(ss.potential_energy)


-24377.96102599183 kJ/mol


In [10]:
thermostate = ThermodynamicState(rtf.REST_system, temperature=300*unit.kelvin)
integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = thermostate.create_context(integrator)
ss.apply_to_context(context)
ss.update_from_context(context)

In [11]:
ss.potential_energy

Quantity(value=-24406.880053823006, unit=kilojoule/mole)

In [15]:
og_forces = [q[1] for q in compute_potential_components(og_context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [16]:
new_forces = [q[1] for q in compute_potential_components(context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [17]:
og_forces[0] - new_forces[0] #bond

0.0

In [18]:
og_forces[1] - new_forces[1] #angle

-1.1102230246251565e-16

In [19]:
og_forces[2] - new_forces[2] #torsion

-3.298235728976806e-06

In [20]:
np.array(og_forces[3:]).sum() - np.array(new_forces[3:]).sum() #nonbonded

11.593858174837806

In [21]:
og_forces

[0.03455176988667383,
 0.6071341287747579,
 3.2298440541507567,
 -9777.175232374319,
 0.0]

In [22]:
new_forces

[0.03455176988667383,
 0.607134128774758,
 3.2298473523864857,
 -9798.93716972444,
 -96.90382601250437,
 107.07190518778819,
 0.0]

## test the energy discrepancy of the alanine dipeptide in solvent with NoCutoff

In [24]:
from openmmtools.testsystems import AlanineDipeptideExplicit
ala = AlanineDipeptideExplicit(useDispersionCorrection=False, nonbondedMethod=app.NoCutoff)

In [25]:
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]

In [26]:
ala.system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b2e79900ab0> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b2e79900de0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b2e799000c0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b2ecfcd7c60> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x2b2ecfcd7cf0> >]

In [27]:
ala.system.removeForce(4) #remove CMM

In [28]:
rtf = RESTTopologyFactory(system = ala.system, solute_region=rest_atoms, use_dispersion_correction=False)

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.


In [29]:
ss = SamplerState(positions=ala.positions)

In [30]:
og_thermostate = ThermodynamicState(ala.system, temperature=300*unit.kelvin)
og_integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
og_context = og_thermostate.create_context(og_integrator)
ss.apply_to_context(og_context)
ss.update_from_context(og_context)
print(ss.potential_energy)


-24519.86947259385 kJ/mol


In [31]:
thermostate = ThermodynamicState(rtf.REST_system, temperature=300*unit.kelvin)
integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = thermostate.create_context(integrator)
ss.apply_to_context(context)
ss.update_from_context(context)

In [32]:
ss.potential_energy

Quantity(value=-24519.869461925562, unit=kilojoule/mole)

In [33]:
og_forces = [q[1] for q in compute_potential_components(og_context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [34]:
new_forces = [q[1] for q in compute_potential_components(context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [35]:
og_forces[0] - new_forces[0] #bond

0.0

In [36]:
og_forces[1] - new_forces[1] #angle

0.0

In [37]:
og_forces[2] - new_forces[2] #torsion

-3.298235663695692e-06

In [38]:
np.array(og_forces[3:]).sum() - np.array(new_forces[3:]).sum() #nonbonded

-9.787581802811474e-07

In [39]:
og_forces

[0.034551769886566816,
 0.6071341287752139,
 3.229844054150822,
 -9834.068036693374,
 0.0]

In [40]:
new_forces

[0.034551769886566816,
 0.6071341287752139,
 3.2298473523864857,
 -9793.502338902084,
 -147.6376020003182,
 107.07190518778836,
 0.0]